In [1]:
import findspark
import math
import numpy as np 
import pandas as pd 
from pyspark.sql.functions import isnan, when, count, col, isnull, asc, desc, mean
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read.format("csv").option("inferschema","True").option("header","True").option("sep",",").load("netflix_list.csv")

In [2]:
display(df)

DataFrame[imdb_id: string, title: string, certificate: string, startYear: int, type: string, origin_country: string, language: string, rating: double, numVotes: double, genres: string]

In [3]:
df.show(5)

+----------+--------------------+-----------+---------+-----+--------------+--------+------+--------+--------------------+
|   imdb_id|               title|certificate|startYear| type|origin_country|language|rating|numVotes|              genres|
+----------+--------------------+-----------+---------+-----+--------------+--------+------+--------+--------------------+
| tt0993840|    Army of the Dead|         18|     2021|movie| United States| English|   5.8|110780.0| Action,Crime,Horror|
| tt6111574|The Woman in the ...|         18|     2021|movie| United States| English|   5.7| 46703.0| Crime,Drama,Mystery|
| tt7979580|The Mitchells vs ...|         13|     2021|movie| United States| English|   7.8| 55762.0|Adventure,Animati...|
| tt7084386|        Blue Miracle|         13|     2021|movie| United States| English|   6.7|  2962.0|Adventure,Biograp...|
|tt10962368| Things Heard & Seen|         16|     2021|movie| United States| English|   5.3| 20762.0|Drama,Horror,Mystery|
+----------+----

In [4]:
df.columns


['imdb_id',
 'title',
 'certificate',
 'startYear',
 'type',
 'origin_country',
 'language',
 'rating',
 'numVotes',
 'genres']

In [5]:
df.sort("numVotes","rating").show(truncate=False)
#df = df.sort(col("numVotes"),col("rating"))

+----------+--------------------------------+-----------+---------+-----+--------------+--------+------+--------+--------------------------+
|imdb_id   |title                           |certificate|startYear|type |origin_country|language|rating|numVotes|genres                    |
+----------+--------------------------------+-----------+---------+-----+--------------+--------+------+--------+--------------------------+
|tt6338498 |Vivo                            |PG         |2021     |movie|Canada        |English |null  |null    |Adventure,Animation,Comedy|
|tt13575806|Back to the Outback             |null       |2021     |movie|United States |English |null  |null    |Adventure,Animation,Comedy|
|tt9421570 |The Guilty                      |R          |2021     |movie|United States |English |null  |null    |Crime,Drama,Thriller      |
|tt11286314|Don't Look Up                   |null       |2021     |movie|United States |English |null  |null    |Comedy                    |
|tt10293406|T

In [6]:
'''Find the mean of the column "rating" '''
mean_ = df.select(mean(col('rating'))).take(1)[0][0]
mean_ = math.ceil(mean_)
print(mean_)

7


In [7]:
'''Find the mean of the column "rating" '''
mean1_ = df.select(mean(col('numVotes'))).take(1)[0][0]
mean1_ = math.ceil(mean1_)
print(mean1_)

18644


In [8]:
import pyspark.sql.functions as F
df = df.withColumn("rating/numVotes",  (F.col("rating") / F.col("numVotes")))

In [9]:
import pandas
df.select("rating/numVotes").toPandas()

,rating/numVotes
0,0.000052
1,0.000122
2,0.000140
3,0.002262
4,0.000255
...,...
230,0.250000
231,0.178788
232,0.070408
233,0.187879


In [10]:
print(df.select(['title']).collect()[0]) #first(),show(0) etc.

Row(title='Army of the Dead')


In [11]:
df.select('title','genres').where(df['genres'].isin(['Crime,Drama,Mystery'])).show()

+--------------------+-------------------+
|               title|             genres|
+--------------------+-------------------+
|The Woman in the ...|Crime,Drama,Mystery|
|      I Am All Girls|Crime,Drama,Mystery|
+--------------------+-------------------+



In [12]:
df.select('certificate').distinct().show()

+-----------+
|certificate|
+-----------+
|          7|
|         16|
|       null|
|         PG|
|         18|
|          U|
|        All|
|          A|
|          R|
|          G|
|         13|
|      PG-13|
+-----------+



In [13]:
df.select('type').distinct().show()

+-----+
| type|
+-----+
|movie|
+-----+



In [14]:
df.select('language').distinct().show()

+--------+
|language|
+--------+
| English|
+--------+



In [15]:
df.crosstab('title','certificate').select('title_certificate','7','16','null','PG','18','U','All','A','R','G','13','PG-13').show()

+--------------------+---+---+----+---+---+---+---+---+---+---+---+-----+
|   title_certificate|  7| 16|null| PG| 18|  U|All|  A|  R|  G| 13|PG-13|
+--------------------+---+---+----+---+---+---+---+---+---+---+---+-----+
|Rise of the Teena...|  0|  0|   1|  0|  0|  0|  0|  0|  0|  0|  0|    0|
|       Farewell Amor|  0|  0|   1|  0|  0|  0|  0|  0|  0|  0|  0|    0|
|The App That Stol...|  0|  0|   1|  0|  0|  0|  0|  0|  0|  0|  0|    0|
|  Single All the Way|  0|  0|   1|  0|  0|  0|  0|  0|  0|  0|  0|    0|
|Happiness Ever After|  0|  0|   1|  0|  0|  0|  0|  0|  0|  0|  0|    0|
|Secrets of the Sa...|  0|  0|   0|  0|  0|  0|  0|  0|  0|  0|  1|    0|
|Bob Ross: Happy A...|  0|  0|   1|  0|  0|  0|  0|  0|  0|  0|  0|    0|
|Mucho Mucho Amor:...|  0|  0|   0|  0|  0|  0|  0|  0|  0|  0|  1|    0|
|The Last Days of ...|  0|  0|   0|  0|  1|  0|  0|  0|  0|  0|  0|    0|
|Have a Good Trip:...|  0|  0|   1|  0|  0|  0|  0|  0|  0|  0|  0|    0|
|United Nation Thr...|  0|  0|   1|  0

In [16]:
df.select('genres').show()

+--------------------+
|              genres|
+--------------------+
| Action,Crime,Horror|
| Crime,Drama,Mystery|
|Adventure,Animati...|
|Adventure,Biograp...|
|Drama,Horror,Mystery|
|Drama,Sci-Fi,Thri...|
|              Comedy|
| Crime,Drama,Mystery|
|Adventure,Animati...|
|Biography,Drama,H...|
|Action,Adventure,...|
|            Thriller|
|Action,Adventure,...|
|      Horror,Mystery|
|Crime,Documentary...|
|Action,Adventure,...|
|  Comedy,Drama,Music|
|       Comedy,Family|
|Action,Adventure,...|
|Action,Adventure,...|
+--------------------+
only showing top 20 rows



In [17]:
df.crosstab('type','genres').select('type_genres','Action,Crime,Horror','Crime,Drama,Mystery').show()

+-----------+-------------------+-------------------+
|type_genres|Action,Crime,Horror|Crime,Drama,Mystery|
+-----------+-------------------+-------------------+
|      movie|                  2|                  2|
+-----------+-------------------+-------------------+



In [18]:
from pyspark.sql.functions import *


In [19]:
df = df.na.fill({'rating':mean_}).limit()


TypeError: limit() missing 1 required positional argument: 'num'

In [ ]:
df = df.fillna({'numVotes':mean1_})

In [ ]:
from pyspark.sql.functions import col
df.filter(col("genres").contains("Crime")).show()


In [ ]:
crime_movie.count()

In [ ]:
split_col = split(df['genres'], ',')

In [ ]:
df2 = df.withColumn('genre_type_1', split_col.getItem(0)) \
       .withColumn('genre_type_2', split_col.getItem(1)) \
       .withColumn('genre_type_3', split_col.getItem(2))
df2.show(truncate=False)   

In [ ]:
df2.select('genre_type_1','genre_type_2','genre_type_3').show()

In [ ]:
pd_df = pd.read_csv("netflix_list.csv")
pd_df.certificate =pd_df.certificate.astype(str)

In [ ]:
pd_df.info()

In [ ]:
pd_df =df.na.fill({'numVotes':mean1_})

In [ ]:
pd_df.show()

In [ ]:
pd_df = pd_df.na.fill({'rating':mean_}).show()


In [ ]:
df = df.withColumn("rating/numVotes", df.rating/df.numVotes)

In [ ]:
df.select("rating/numVotes").show()

In [ ]:
from pyspark.sql.functions import *
df.filter(df['rating/numVotes'].isNull()).count()

In [ ]:
df.printSchema()

In [ ]:
df.orderBy('rating/numVotes').tail(1)